# LSTM Bitcoin Prediction

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# load data
raw_df = pd.read_csv('bitcoin_20190307_per_day.csv')

In [3]:
raw_df.shape

(3157, 7)

In [4]:
raw_df['time'] = pd.to_datetime(raw_df.time, unit='s')

In [5]:
raw_df.isnull().sum()

close         0
high          0
low           0
open          0
time          0
volumefrom    0
volumeto      0
dtype: int64

In [6]:
raw_df.head()

,close,high,low,open,time,volumefrom,volumeto
0,0.04951,0.04951,0.04951,0.04951,2010-07-17,20.00,0.9902
1,0.08584,0.08585,0.05941,0.04951,2010-07-18,75.01,5.0900
2,0.08080,0.09307,0.07723,0.08584,2010-07-19,574.00,49.6600
3,0.07474,0.08181,0.07426,0.08080,2010-07-20,262.00,20.5900
4,0.07921,0.07921,0.06634,0.07474,2010-07-21,575.00,42.2600


In [7]:
raw_df.tail()

,close,high,low,open,time,volumefrom,volumeto
3152,3731.28,3831.01,3705.14,3812.31,2019-03-04,39278.22,1.472457e+08
3153,3874.18,3893.52,3720.49,3731.28,2019-03-05,43262.30,1.650623e+08
3154,3874.98,3916.29,3839.39,3874.18,2019-03-06,33536.95,1.295205e+08
3155,3882.61,3911.76,3853.20,3874.98,2019-03-07,37619.38,1.462017e+08
3156,3922.66,3930.87,3875.90,3882.61,2019-03-08,24735.74,9.649103e+07


In [8]:
# select after 2017-01-01
selected_df = raw_df[raw_df.time >= '2017-01-01']

In [ ]:
# raw_df[raw_df.time == '2017-11-01']

----

In [9]:
df = selected_df.drop(columns='time', axis=1)

In [10]:
df.head()

,close,high,low,open,volumefrom,volumeto
2360,995.44,1001.61,956.10,963.38,41148.52,4.057092e+07
2361,1017.05,1031.68,990.20,995.44,64951.63,6.603807e+07
2362,1033.30,1035.47,1006.53,1017.05,54788.01,5.608587e+07
2363,1135.41,1148.54,1022.32,1033.30,156272.17,1.705846e+08
2364,989.35,1150.63,874.53,1135.41,240007.84,2.448393e+08


In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
arr = scaler.fit_transform(df)

In [12]:
df = pd.DataFrame(arr)

In [13]:
df.columns=['close', 'high', 'low', 'open', 'volumefrom', 'volumeto']

In [14]:
df

,close,high,low,open,volumefrom,volumeto
0,0.011315,0.009199,0.012023,0.009587,0.062076,0.004868
1,0.012479,0.010778,0.013916,0.011314,0.111171,0.008953
2,0.013355,0.010977,0.014823,0.012479,0.090208,0.007356
3,0.018857,0.016914,0.015700,0.013354,0.299527,0.025719
4,0.010987,0.017024,0.007494,0.018855,0.472238,0.037627
5,0.005427,0.010554,0.006274,0.010986,0.377943,0.027094
6,0.005573,0.003908,0.003726,0.005427,0.246706,0.016490
7,0.006219,0.005760,0.007567,0.005573,0.135828,0.009636
8,0.006161,0.004413,0.007293,0.006219,0.106279,0.007337
9,0.006408,0.004458,0.008360,0.006161,0.087984,0.006163


## LSTM Data Processing 

In [15]:
# build Data, use last 30 days to predict next 5 days
def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]['close']))
  return np.array(X_train), np.array(Y_train)

In [16]:
# shuffle the data, and random seed is 10
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [17]:
# split training data and validation data
def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

### 分割預設值: 30天預測後5天

In [ ]:
X_train, Y_train = buildTrain(df, 30, 5)

In [ ]:
X_train, Y_train = shuffle(X_train, Y_train)

In [ ]:
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [ ]:
print('X_train=', X_train.shape)
print('Y_train=', Y_train.shape)
print('X_val=', X_val.shape)
print('Y_val=', Y_val.shape)

---

## One to One 

In [19]:
model = Sequential()
model.add(LSTM(10, input_length=1, input_dim=6,return_sequences=True))
# output shape: (1, 1)
model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 10)             680       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              11        
Total params: 691
Trainable params: 691
Non-trainable params: 0
_________________________________________________________________


C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 6), return_sequences=True)`
  


In [20]:
X_train, Y_train = buildTrain(df, 1, 1)

In [21]:
X_train, Y_train = shuffle(X_train, Y_train)

In [22]:
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [23]:
# from 2 dimmension to 3 dimension, model output: (None, 1, 1)
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

In [ ]:
print('X_train=', X_train.shape)
print('Y_train=', Y_train.shape)
print('X_val=', X_val.shape)
print('Y_val=', Y_val.shape)

In [24]:
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=100, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Instructions for updating:
Use tf.cast instead.
Train on 716 samples, validate on 79 samples
Epoch 1/100
716/716 [==============================] - 1s 950us/step - loss: 0.0931 - val_loss: 0.0834
Epoch 2/100
716/716 [==============================] - 0s 13us/step - loss: 0.0820 - val_loss: 0.0728
Epoch 3/100
716/716 [==============================] - 0s 14us/step - loss: 0.0720 - val_loss: 0.0632
Epoch 4/100
716/716 [==============================] - 0s 11us/step - loss: 0.0627 - val_loss: 0.0545
Epoch 5/100
716/716 [==============================] - 0s 13us/step - loss: 0.0545 - val_loss: 0.0468
Epoch 6/100
716/716 [==============================] - 0s 14us/step - loss: 0.0471 - val_loss: 0.0399
Epoch 7/100
716/716 [==============================] - 0s 16us/step - loss: 0.0407 - val_loss: 0.0337
Epoch 8/100
716/716 [==============================] - 0s 14us/step - loss: 0.0347 - val_loss: 0.0285
Epoch 9/100
716/716 [==============================] - 0s 13us/step - loss: 0.0298 - val_l

Epoch 76/100
716/716 [==============================] - 0s 14us/step - loss: 6.3048e-04 - val_loss: 7.3984e-04
Epoch 77/100
716/716 [==============================] - 0s 15us/step - loss: 6.2492e-04 - val_loss: 7.3407e-04
Epoch 78/100
716/716 [==============================] - 0s 15us/step - loss: 6.1983e-04 - val_loss: 7.3036e-04
Epoch 79/100
716/716 [==============================] - 0s 13us/step - loss: 6.1517e-04 - val_loss: 7.2346e-04
Epoch 80/100
716/716 [==============================] - 0s 13us/step - loss: 6.1053e-04 - val_loss: 7.1991e-04
Epoch 81/100
716/716 [==============================] - 0s 13us/step - loss: 6.0598e-04 - val_loss: 7.1315e-04
Epoch 82/100
716/716 [==============================] - 0s 12us/step - loss: 6.0159e-04 - val_loss: 7.0652e-04
Epoch 83/100
716/716 [==============================] - 0s 13us/step - loss: 5.9716e-04 - val_loss: 7.0223e-04
Epoch 84/100
716/716 [==============================] - 0s 13us/step - loss: 5.9403e-04 - val_loss: 7.0282e-04
E

## Many to One (30 to 1)

In [25]:
model = Sequential()
model.add(LSTM(10, input_length=30, input_dim=6))
# output shape: (1, 1)
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10)                680       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 691
Trainable params: 691
Non-trainable params: 0
_________________________________________________________________


C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(30, 6))`
  


In [26]:
X_train, Y_train = buildTrain(df, 30, 1)

In [ ]:
print('X_train=', X_train.shape)
print('Y_train=', Y_train.shape)
print('X_val=', X_val.shape)
print('Y_val=', Y_val.shape)

In [27]:
X_train, Y_train = shuffle(X_train, Y_train)

In [28]:
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [29]:
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
hist = model.fit(X_train, Y_train, epochs=100, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Train on 690 samples, validate on 76 samples
Epoch 1/100
690/690 [==============================] - 1s 1ms/step - loss: 0.0633 - val_loss: 0.0566
Epoch 2/100
690/690 [==============================] - 0s 76us/step - loss: 0.0304 - val_loss: 0.0248
Epoch 3/100
690/690 [==============================] - 0s 78us/step - loss: 0.0120 - val_loss: 0.0104
Epoch 4/100
690/690 [==============================] - 0s 78us/step - loss: 0.0076 - val_loss: 0.0086
Epoch 5/100
690/690 [==============================] - 0s 76us/step - loss: 0.0083 - val_loss: 0.0078
Epoch 6/100
690/690 [==============================] - 0s 75us/step - loss: 0.0068 - val_loss: 0.0061
Epoch 7/100
690/690 [==============================] - 0s 82us/step - loss: 0.0048 - val_loss: 0.0057
Epoch 8/100
690/690 [==============================] - 0s 76us/step - loss: 0.0040 - val_loss: 0.0055
Epoch 9/100
690/690 [==============================] - 0s 77us/step - loss: 0.0035 - val_loss: 0.0048
Epoch 10/100
690/690 [================

690/690 [==============================] - 0s 73us/step - loss: 0.0011 - val_loss: 0.0017
Epoch 81/100
690/690 [==============================] - 0s 75us/step - loss: 0.0011 - val_loss: 0.0016
Epoch 82/100
690/690 [==============================] - 0s 73us/step - loss: 0.0010 - val_loss: 0.0016
Epoch 83/100
690/690 [==============================] - 0s 75us/step - loss: 0.0010 - val_loss: 0.0016
Epoch 84/100
690/690 [==============================] - 0s 78us/step - loss: 0.0010 - val_loss: 0.0016
Epoch 85/100
690/690 [==============================] - 0s 78us/step - loss: 0.0010 - val_loss: 0.0016
Epoch 86/100
690/690 [==============================] - 0s 78us/step - loss: 0.0010 - val_loss: 0.0016
Epoch 87/100
690/690 [==============================] - 0s 78us/step - loss: 0.0010 - val_loss: 0.0016
Epoch 88/100
690/690 [==============================] - 0s 76us/step - loss: 0.0010 - val_loss: 0.0016
Epoch 89/100
690/690 [==============================] - 0s 77us/step - loss: 9.9697e-0

## One to Many (1 to 5)

In [30]:
model = Sequential()
model.add(LSTM(10, input_length=1, input_dim=6))
# output shape: (5, 1)
model.add(Dense(1))
model.add(RepeatVector(5))
model.compile(loss="mse", optimizer="adam")
model.summary()

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 6))`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10)                680       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 1)              0         
Total params: 691
Trainable params: 691
Non-trainable params: 0
_________________________________________________________________


In [31]:
X_train, Y_train = buildTrain(df, 1, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [32]:
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

In [33]:
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=100, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Train on 712 samples, validate on 79 samples
Epoch 1/100
712/712 [==============================] - 1s 1ms/step - loss: 0.1121 - val_loss: 0.0985
Epoch 2/100
712/712 [==============================] - 0s 13us/step - loss: 0.0973 - val_loss: 0.0849
Epoch 3/100
712/712 [==============================] - 0s 14us/step - loss: 0.0842 - val_loss: 0.0727
Epoch 4/100
712/712 [==============================] - 0s 13us/step - loss: 0.0722 - val_loss: 0.0620
Epoch 5/100
712/712 [==============================] - 0s 14us/step - loss: 0.0619 - val_loss: 0.0526
Epoch 6/100
712/712 [==============================] - 0s 14us/step - loss: 0.0526 - val_loss: 0.0446
Epoch 7/100
712/712 [==============================] - 0s 13us/step - loss: 0.0448 - val_loss: 0.0378
Epoch 8/100
712/712 [==============================] - 0s 14us/step - loss: 0.0383 - val_loss: 0.0321
Epoch 9/100
712/712 [==============================] - 0s 14us/step - loss: 0.0327 - val_loss: 0.0274
Epoch 10/100
712/712 [================

712/712 [==============================] - 0s 13us/step - loss: 0.0018 - val_loss: 0.0015
Epoch 81/100
712/712 [==============================] - 0s 13us/step - loss: 0.0017 - val_loss: 0.0015
Epoch 82/100
712/712 [==============================] - 0s 15us/step - loss: 0.0017 - val_loss: 0.0015
Epoch 83/100
712/712 [==============================] - 0s 13us/step - loss: 0.0017 - val_loss: 0.0014
Epoch 84/100
712/712 [==============================] - 0s 12us/step - loss: 0.0017 - val_loss: 0.0014
Epoch 85/100
712/712 [==============================] - 0s 13us/step - loss: 0.0017 - val_loss: 0.0014
Epoch 86/100
712/712 [==============================] - 0s 13us/step - loss: 0.0017 - val_loss: 0.0014
Epoch 87/100
712/712 [==============================] - 0s 13us/step - loss: 0.0017 - val_loss: 0.0014
Epoch 88/100
712/712 [==============================] - 0s 15us/step - loss: 0.0017 - val_loss: 0.0014
Epoch 89/100
712/712 [==============================] - 0s 14us/step - loss: 0.0017 - 

## Many to Many (5 to 5)

In [34]:
model = Sequential()
model.add(LSTM(10, input_length=5, input_dim=6, return_sequences=True))
# output shape: (5, 1)
model.add(TimeDistributed(Dense(1)))
model.compile(loss="mse", optimizer="adam")
model.summary()

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(5, 6), return_sequences=True)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 5, 10)             680       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 1)              11        
Total params: 691
Trainable params: 691
Non-trainable params: 0
_________________________________________________________________


In [35]:
X_train, Y_train = buildTrain(df, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

In [36]:
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

In [37]:
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

Train on 709 samples, validate on 78 samples
Epoch 1/1000
709/709 [==============================] - 1s 1ms/step - loss: 0.1156 - val_loss: 0.0763
Epoch 2/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0858 - val_loss: 0.0533
Epoch 3/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0614 - val_loss: 0.0358
Epoch 4/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0426 - val_loss: 0.0237
Epoch 5/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0299 - val_loss: 0.0164
Epoch 6/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0219 - val_loss: 0.0130
Epoch 7/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0184 - val_loss: 0.0121
Epoch 8/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0169 - val_loss: 0.0123
Epoch 9/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0166 - val_loss: 0.0124
Epoch 10/1000
709/709 [======

709/709 [==============================] - 0s 25us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 80/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 81/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 82/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 83/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 84/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 85/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 86/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 87/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 88/1000
709/709 [==============================] - 0s 23us/step - loss: 

709/709 [==============================] - 0s 23us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 158/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 159/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 160/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 161/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 162/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 163/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 164/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 165/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 166/1000
709/709 [==============================] - 0s 23us/step

709/709 [==============================] - 0s 25us/step - loss: 0.0022 - val_loss: 0.0018
Epoch 236/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0022 - val_loss: 0.0017
Epoch 237/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0022 - val_loss: 0.0017
Epoch 238/1000
709/709 [==============================] - 0s 27us/step - loss: 0.0022 - val_loss: 0.0017
Epoch 239/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0022 - val_loss: 0.0018
Epoch 240/1000
709/709 [==============================] - 0s 27us/step - loss: 0.0022 - val_loss: 0.0017
Epoch 241/1000
709/709 [==============================] - 0s 26us/step - loss: 0.0022 - val_loss: 0.0017
Epoch 242/1000
709/709 [==============================] - 0s 27us/step - loss: 0.0021 - val_loss: 0.0017
Epoch 243/1000
709/709 [==============================] - 0s 26us/step - loss: 0.0021 - val_loss: 0.0017
Epoch 244/1000
709/709 [==============================] - 0s 25us/step

709/709 [==============================] - 0s 24us/step - loss: 0.0021 - val_loss: 0.0016
Epoch 314/1000
709/709 [==============================] - 0s 26us/step - loss: 0.0021 - val_loss: 0.0017
Epoch 315/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0021 - val_loss: 0.0017
Epoch 316/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0021 - val_loss: 0.0016
Epoch 317/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0021 - val_loss: 0.0016
Epoch 318/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0021 - val_loss: 0.0016
Epoch 319/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0021 - val_loss: 0.0016
Epoch 320/1000
709/709 [==============================] - 0s 23us/step - loss: 0.0021 - val_loss: 0.0016
Epoch 321/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0021 - val_loss: 0.0016
Epoch 322/1000
709/709 [==============================] - 0s 25us/step

709/709 [==============================] - 0s 23us/step - loss: 0.0020 - val_loss: 0.0015
Epoch 392/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0020 - val_loss: 0.0016
Epoch 393/1000
709/709 [==============================] - 0s 26us/step - loss: 0.0020 - val_loss: 0.0015
Epoch 394/1000
709/709 [==============================] - 0s 27us/step - loss: 0.0020 - val_loss: 0.0015
Epoch 395/1000
709/709 [==============================] - 0s 26us/step - loss: 0.0020 - val_loss: 0.0016
Epoch 396/1000
709/709 [==============================] - 0s 28us/step - loss: 0.0020 - val_loss: 0.0015
Epoch 397/1000
709/709 [==============================] - 0s 27us/step - loss: 0.0020 - val_loss: 0.0015
Epoch 398/1000
709/709 [==============================] - 0s 25us/step - loss: 0.0020 - val_loss: 0.0016
Epoch 399/1000
709/709 [==============================] - 0s 24us/step - loss: 0.0020 - val_loss: 0.0015
Epoch 400/1000
709/709 [==============================] - 0s 23us/step